<a href="https://colab.research.google.com/github/asandy1023/devops-demo/blob/master/taiwan_aicup_Spring_bert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install transformers

import torch
import json
import sys
import re
import pandas as pd
import numpy as np
from transformers import BertTokenizer, AutoTokenizer, AutoModel
from IPython.display import clear_output



PRETRAINED_MODEL_NAME = "bert-base-chinese"  # Chinese BERT-BASE pre-trained model
# The tokenizer used by the model 
tokenizer = AutoTokenizer.from_pretrained('bert-base-chinese')
embedding = AutoModel.from_pretrained('bert-base-chinese')

clear_output()
print("PyTorch 版本：", torch.__version__)

PyTorch 版本： 1.8.1+cu101


In [ ]:
# preprocessing
def Sentence_Segmentation(line):
  temp = re.split(r"([。])",line)
  # period join
  for i in range( len(temp) ):
    if temp[i] == '。':
      temp[i-1] = temp[i-1] + temp[i]
  # delete
  r_lines = [line for line in temp if not line in ['', '。']]
  return r_lines

In [ ]:
# List擴充, data請輸入未斷詞單一字串
def List_Append(vocs, txt):
  # 斷句
  sents = Sentence_Segmentation(txt)
  for sent in sents:
    # BERT斷詞
    sent_token = []
    # 太長就分開
    while len(sent) > 500 :
      sent_token = sent_token + tokenizer.encode(sent[:500])
      sent = sent[500:]
    sent_token = sent_token + tokenizer.encode(sent[:500])

    for word in sent:
      if word not in vocs:
        vocs.append(word)
  return vocs

In [ ]:
csv_file = pd.read_csv("Train_risk_classification_ans.csv", header=0,encoding='utf-8',dtype=str)
csv_datas = pd.DataFrame(csv_file)
print('有',len(csv_datas),'組資料')
csv_datas[:5]['text']

有 346 組資料


0    個管師：這個月還好嗎？民眾：蛤？個管師：這個月還好嗎？民眾：這個月還好。個管師：還好，還可以...
1    個管師：所以你這個月還OK？民眾：還OK啊。個管師：那有固定伴侶嗎？民眾：沒有。個管師：你一...
2    民眾：啊就在忙裝潢啊～幹嘛的。個管師：嗯。民眾：對。個管師：所以都在熬夜？民眾：對，都一直熬...
3    個管師：對啊，你是說你要幫，我幫你給這個？你說小花跟你。民眾：現在是狀況是還好。個管師：嗯。...
4    個管師：好喔，過去這一個月還好嗎？民眾：恩，還可以。個管師：那今天是請假？民眾：沒有，晚上上...
Name: text, dtype: object

In [ ]:
# Create vocab_list and embeddings for classification datasets
vocabs = ['[PAD]']

for text in csv_datas['text']:
  vocabs = List_Append(vocabs, text)

print('vocabs共 %d 筆資料' % ( len(vocabs) ) )
print(vocabs)

vocabs共 2381 筆資料
['[PAD]', '個', '管', '師', '：', '這', '月', '還', '好', '嗎', '？', '民', '眾', '蛤', '。', '，', '可', '以', '有', '性', '行', '為', '所', '是', '跟', '固', '定', '伴', '侶', '對', '喔', '阿', '你', '囉', '本', '來', '就', '只', '比', '較', '忙', '很', '少', '約', '一', '直', '都', '那', '恩', '诶', '不', '後', '用', '他', '的', '然', '扮', '相', '沒', '了', '什', '麼', '身', '分', '吃', 'P', 'r', 'E', '我', '朋', '友', '們', '做', '時', '候', '現', '在', '1', '2', '之', '前', '夠', '話', '會', '拿', '剛', '因', '也', 'O', 'K', '要', '說', '戴', '套', '二', '擇', '樣', '子', '啊', '能', '怕', '如', '果', '方', '炮', '講', '外', '面', '幹', '嘛', '信', '多', '久', '五', '、', '六', '年', '吧', '啦', '七', '八', '次', '曉', '得', '想', '耶', '主', '動', '去', '但', '基', '上', '忘', '記', '全', '程', '應', '該', '求', '當', '哪', '哈', '喜', '歡', '反', '而', '覺', '⋯', '哼', '任', '務', '型', '嗯', '導', '致', '才', '幾', '組', '兩', '正', '保', '險', '自', '己', '關', '係', '藥', '天', '或', '再', '安', '感', '無', '差', '嘿', '擔', '心', '害', '效', '情', '境', '問', '下', '顆', '隔', '十', '四', '小', '到', '甚', '至', '護', '給', '看', '證', '明'

In [ ]:
##########################################

In [ ]:
# Load the vocab.json file
vocabs_file = open('vocab.json')
vocabs_dict = json.load(vocabs_file)
print(len(vocabs_dict))
print(vocabs_dict)

2392
{'[PAD]': 0, '：': 1, '這': 2, '個': 3, '月': 4, '還': 5, '好': 6, '嗎': 7, '？': 8, '蛤': 9, '，': 10, '可': 11, '以': 12, '有': 13, '性': 14, '行': 15, '為': 16, '所': 17, '是': 18, '跟': 19, '固': 20, '定': 21, '伴': 22, '侶': 23, '對': 24, '喔': 25, '阿': 26, '你': 27, '囉': 28, '本': 29, '來': 30, '就': 31, '只': 32, '比': 33, '較': 34, '忙': 35, '很': 36, '少': 37, '約': 38, '一': 39, '直': 40, '都': 41, '那': 42, '恩': 43, '诶': 44, '不': 45, '後': 46, '用': 47, '他': 48, '的': 49, '然': 50, '扮': 51, '相': 52, '沒': 53, '了': 54, '什': 55, '麼': 56, '身': 57, '分': 58, '吃': 59, 'P': 60, 'r': 61, 'E': 62, '我': 63, '朋': 64, '友': 65, '們': 66, '做': 67, '時': 68, '候': 69, '現': 70, '在': 71, '1': 72, '2': 73, '之': 74, '前': 75, '夠': 76, '話': 77, '會': 78, '拿': 79, '剛': 80, '因': 81, '也': 82, 'O': 83, 'K': 84, '要': 85, '說': 86, '戴': 87, '套': 88, '二': 89, '擇': 90, '樣': 91, '子': 92, '啊': 93, '能': 94, '怕': 95, '如': 96, '果': 97, '方': 98, '炮': 99, '講': 100, '外': 101, '面': 102, '幹': 103, '嘛': 104, '信': 105, '多': 106, '久': 107, '五': 108, '、': 109, 

In [ ]:
# Create vocab_list and embeddings of classification dataset and cut them into smaller words
vocabs = ['[PAD]']

for times,text in enumerate(vocabs_dict):
  if times <= 0 :# Remove vocab.json in '['P'A'D']'
    continue
  vocabs = List_Append(vocabs, text)

print('vocabs共 %d 筆資料' % ( len(vocabs) ) )
print(vocabs)

vocabs共 2392 筆資料
['[PAD]', '：', '這', '個', '月', '還', '好', '嗎', '？', '蛤', '，', '可', '以', '有', '性', '行', '為', '所', '是', '跟', '固', '定', '伴', '侶', '對', '喔', '阿', '你', '囉', '本', '來', '就', '只', '比', '較', '忙', '很', '少', '約', '一', '直', '都', '那', '恩', '诶', '不', '後', '用', '他', '的', '然', '扮', '相', '沒', '了', '什', '麼', '身', '分', '吃', 'P', 'r', 'E', '我', '朋', '友', '們', '做', '時', '候', '現', '在', '1', '2', '之', '前', '夠', '話', '會', '拿', '剛', '因', '也', 'O', 'K', '要', '說', '戴', '套', '二', '擇', '樣', '子', '啊', '能', '怕', '如', '果', '方', '炮', '講', '外', '面', '幹', '嘛', '信', '多', '久', '五', '、', '六', '年', '吧', '啦', '七', '八', '次', '曉', '得', '想', '耶', '主', '動', '去', '但', '基', '上', '忘', '記', '全', '程', '應', '該', '求', '當', '哪', '哈', '喜', '歡', '反', '而', '覺', '⋯', '哼', '任', '務', '型', '嗯', '導', '致', '才', '幾', '組', '兩', '正', '保', '險', '自', '己', '關', '係', '藥', '天', '或', '再', '安', '感', '無', '差', '嘿', '擔', '心', '害', '效', '情', '境', '問', '下', '顆', '隔', '十', '四', '小', '到', '甚', '至', '護', '給', '看', '證', '明', '文', '件', '從', '事', '其'

In [ ]:
# embeddings

embeddings = np.array([np.zeros(768)])

pop_index_list = []

vocabs_len = len(vocabs)

for i in range(vocabs_len):
  if i == 0 :
    continue
  
  encoded_word = tokenizer.encode(vocabs[i])
  try:
    # Convert
    inputs = torch.tensor([encoded_word])
    masks = torch.tensor([[1.0 ,1.0 ,1.0]])
    embedded = embedding(inputs, attention_mask=masks)
    embeddings = np.row_stack((embeddings, embedded[0][0][1].detach().numpy()))
  except:
    pop_index_list.append(i)

  if i%50 == 0:
    clear_output()
    print(i*100/vocabs_len,'%')

print('有',len(pop_index_list),'組資料無法編碼or剃除')
for index in pop_index_list:
  try:
    print('自動剃除index:',index,vocabs[index])
    del vocabs[index]
  except:
    print('無法剃除:',index)

print('len(vocabs)=',len(vocabs))
print('embeddings.shape=',embeddings.shape)
if len(embeddings) == len(vocabs):
  print('資料一致')
else:
  print('資料不一致')

98.24414715719064 %
有 1 組資料無法編碼or剃除
自動剃除index: 2278 　
len(vocabs)= 2391
embeddings.shape= (2391, 768)
資料一致


In [ ]:
# Convert the morphology of vocabs to a dictionary
vocabs_dict = {}
for i in range(len(vocabs)):
  vocabs_dict[vocabs[i]] = i
len(vocabs_dict)

2391

In [ ]:
# save vocabs_dict
json_file_name = 'vocab.json'

with open(json_file_name, 'w', encoding='utf-8') as f:
    json.dump(vocabs_dict, f, ensure_ascii=False)

In [ ]:
# Load the vocab.json file
vocabs_file = open('vocab.json')
vocabs_dict = json.load(vocabs_file)

print('vocab.json共 %d 筆資料' % ( len(vocabs_dict) ) )

vocab.json共 2392 筆資料


In [ ]:
# save embeddings
np.save('embeddings', embeddings)

In [ ]:
# load embeddings
embeddings = np.load('embeddings.npy')
print(embeddings.shape)

(2392, 768)


In [ ]:
# load the Train_qa_ans.json file
QA_file = open('Train_qa_ans.json')
QA_datas = json.load(QA_file)
print('有',len(QA_datas),'組資料')
QA_datas[50]

有 695 組資料


{'answer': 'Ｂ',
 'article_id': 27,
 'id': 51,
 'question': {'choices': [{'label': 'A', 'text': '一個半月'},
   {'label': 'B', 'text': '84天'},
   {'label': 'C', 'text': '兩年'}],
  'stem': '請問要經過多久才能再抽血？'},
 'text': '個管師：這個月還好嗎？民眾：還好。個管師：這個是新伴侶，舊伴侶，還是？民眾：就他幫我用的啊。個管師：你不是沒有跟他在一起了嗎？民眾：沒有啊，就像家人一樣。個管師：但是也不會跟他有性行爲。民眾：完全不會。個管師：對嘛，我記得是這樣，我想説你們爲什麽會一起來？這種感覺很奇怪。民眾：沒有啊，就家人。個管師：但我記得他是你的前任是不是？民眾：對，前男友。個管師：okok，對，因爲我就想説，換人嗎？好像也不是，就本人啊。民眾：因爲感覺他越來越騷。個管師：爲什麽？民眾：不知道欸，他9月好像，好像有點變不好欸。個管師：有嗎？吃藥不是比較好了？哦，好，這個月有約嗎？民眾：一次。個管師：然後無套？民眾：有吃藥。個管師：約的嗎？民眾：固炮。個管師：固炮。就是你現在有個固炮，固多久了？就是你原本跟我講的那個嘛。民眾：嗯。個管師：那也快兩年了。民眾：應該有兩年多啦。個管師：兩年多了。民眾：這是貓嗎？個管師：不是不是，它是個傳送系統，貓這也太恐怖了吧。民眾：上次我聽到我想説貓嗎？個管師：沒有沒有，它就是有一些氣送東西，然後以前這間是病例室，比方説哪裏有需要病例的話，他們就用氣送，後來這間被改爲個管室這樣子。那對方很帥嗎？民眾：還好。個管師：你的菜嗎？他要求無套嗎？民眾：因爲想説他也跟我而已啊。是他這樣講啦，可是我有吃。我，我那時候就會多吃一些。個管師：多吃幾天嗎？還是多吃顆數？民眾：對。可能最多3，1，1，1，1。個管師：哦，這樣可以，這樣可以。但是如果你比如説多吃那顆數就沒差。Ok，好哦。你單子還沒有開出來，我現在問一下，那無套的狀況之下，如果有吃PrEP會不會讓你比較放心？民眾：一點點。個管師：你還是會覺得比較放心？民眾：對。個管師：ok，好那接下來你手上還有幾瓶，因爲你最後一次會回來，你只剩一次回來拿藥的機會了。民眾：而且我上次好像抽血，後來也沒有通知欸，我想説，那就沒有理

In [ ]:
len(vocabs)

2379

In [ ]:
# Add vocabs

New_Start = len(vocabs)

for data in QA_datas:
  #process text
  List_Append(vocabs, data['text'])
  #process Problem Statements
  List_Append(vocabs, data['question']['stem'])
  #Handling Problems Option A, B or C
  for choice in data['question']['choices']:
    List_Append(vocabs, choice['text'])


print('vocabs 共 %d 筆資料，新增了 %d 筆資料' % (len(vocabs), len(vocabs) - New_Start) )

vocabs 共 2395 筆資料，新增了 16 筆資料


In [ ]:
vocabs[60]

'什'

In [ ]:
# embeddings complementary
pop_index_list = []

vocabs_len = len(vocabs)

for i in range(New_Start, vocabs_len):
  encoded_word = tokenizer.encode(vocabs[i])
  try:
    # Convert
    inputs = torch.tensor([encoded_word])
    masks = torch.tensor([[1.0 ,1.0 ,1.0]])
    embedded = embedding(inputs, attention_mask=masks)
    embeddings = np.row_stack((embeddings, embedded[0][0][1].detach().numpy()))
  except:
    pop_index_list.append(i)

print('有',len(pop_index_list),'組資料無法編碼，剃除')
for index in pop_index_list:
  del vocabs[index]

print('len(vocabs)=',len(vocabs))
print('embeddings.shape=',embeddings.shape)
if len(embeddings) == len(vocabs):
  print('資料一致')
else:
  print('資料不一致')

有 1 組資料無法編碼，剃除
len(vocabs)= 2394
embeddings.shape= (2394, 768)
資料一致


In [ ]:
# Convert the morphology of vocabs to a dictionary
vocabs_dict = {}
for i in range(len(vocabs)):
  vocabs_dict[vocabs[i]] = i
print(len(vocabs_dict))

# save vocabs_dict
json_file_name = 'vocab.json'

with open(json_file_name, 'w', encoding='utf-8') as f:
    json.dump(vocabs_dict, f, ensure_ascii=False)

2394


In [ ]:
# save embeddings
np.save('embeddings', embeddings)

In [ ]:
# load vocab.json
vocabs_file = open('vocab.json')
vocabs_dict = json.load(vocabs_file)

print('vocab.json共 %d 筆資料' % ( len(vocabs_dict) ) )

vocab.json共 2394 筆資料


In [ ]:
# load embeddings
embeddings = np.load('embeddings.npy')
print('embeddings.shape = ',embeddings.shape)

embeddings.shape =  (2394, 768)
